In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch

from torch.autograd import Variable
import torch.backends.cudnn as cudnn

from torchvision.utils import save_image

import numpy as np

import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms, models, datasets

In [2]:
'''for epoch in range (200):
  os.mkdir("./output/%03d" % epoch)'''

'for epoch in range (200):\n  os.mkdir("./output/%03d" % epoch)'

# Data

In [3]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def get_backgrounds():
    backgrounds = []
    for file in os.listdir("./images/train"):
        if file.endswith('.jpg'):
            backgrounds.append(plt.imread(os.path.join("./images/train",file)))
    return np.array(backgrounds)
backgrounds = get_backgrounds()


def compose_image(image):
    image = (image > 0).astype(np.float32)
    image = image.reshape([28,28])*255.0
    
    image = np.stack([image,image,image],axis=2)
    
    background = np.random.choice(backgrounds)
    w,h,_ = background.shape
    dw, dh,_ = image.shape
    x = np.random.randint(0,w-dw)
    y = np.random.randint(0,h-dh)
    
    temp = background[x:x+dw, y:y+dh]
    return np.abs(temp-image).astype(np.uint8)


class MNISTM(Dataset):
            
    def __init__(self, train=True,transform=None):
        if train:
            self.data = datasets.MNIST(root='.data/mnist',train=True, download=True)
        else:
            self.data = datasets.MNIST(root='.data/mnist',train=False, download=True)
        self.backgrounds = get_backgrounds()
        self.transform = transform
    def __getitem__(self,index):
        image = np.array(self.data.__getitem__(index)[0])
        target = self.data.__getitem__(index)[1]
        image = compose_image(image)
        #image = Image.fromarray(image.squeeze(), mode="RGB")
        if self.transform is not None:
            image = self.transform(image)
        return image, target
        
    def __len__(self):
        return len(self.data)
    
    
def get_mnistm_loaders(data_aug = False, batch_size=128,test_batch_size=1000):
    if data_aug:
        train_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(64),
            transforms.RandomCrop(64,padding=4),
            transforms.ToTensor()
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(64),
            transforms.ToTensor()
        ])
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(64),
        transforms.ToTensor()
    ])
    kwargs = {}
    train_loader = DataLoader(
        MNISTM(train=True,transform=train_transform),batch_size=batch_size, shuffle=True, drop_last=True)
    train_eval_loader = DataLoader(
        MNISTM(train=True, transform=test_transform),batch_size=test_batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(
        MNISTM(train=False,transform=test_transform),batch_size=test_batch_size, shuffle=False, drop_last=True)
    return train_loader, train_eval_loader, test_loader


def get_mnist_loaders(data_aug = False, batch_size=128,test_batch_size=1000):
    if data_aug:
        train_transform = transforms.Compose(
            [transforms.Resize(64),
            transforms.RandomCrop(64,padding=4),
            transforms.Grayscale(3),
            transforms.ToTensor()
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize(64),
            transforms.Grayscale(3),
            transforms.ToTensor()
        ])
    test_transform = transforms.Compose([
        transforms.Resize(64),
        transforms.Grayscale(3),
        transforms.ToTensor()
    ])
    kwargs = {}

    train_loader = DataLoader(
        datasets.MNIST(root='.data/mnist',train=True, download=True,transform=train_transform),batch_size=batch_size, shuffle=True, drop_last=True)
    train_eval_loader = DataLoader(
        datasets.MNIST(root='.data/mnist',train=True, download=True, transform=test_transform),batch_size=test_batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(
        datasets.MNIST(root='.data/mnist',train=False, download=True, transform=test_transform),batch_size=test_batch_size, shuffle=False, drop_last=True)
    return train_loader, train_eval_loader, test_loader

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


In [4]:
loader_source, mnist_eval_loader, mnist_test_loader = get_mnist_loaders(batch_size=16)
loader_target, mnistm_eval_loader,mnistm_test_loader = get_mnistm_loaders(batch_size=16)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


In [5]:
data_target_iter = iter(loader_source)
target_inputs, target_label = data_target_iter.next()

In [6]:
target_inputs.size()

torch.Size([16, 3, 64, 64])

# Module

## Encoder

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_nc=3):
        super(Encoder, self).__init__()

        # Initial convolution block
        model = [   nn.ReflectionPad2d(3),
                    nn.Conv2d(input_nc, 64, 7),
                    nn.InstanceNorm2d(64),
                    nn.ReLU(inplace=True) ]

        model += [nn.Conv2d(64, 128, 3, stride=2, padding=1),
                  nn.InstanceNorm2d(128),
                  nn.ReLU(inplace=True)]
                  
        model += [nn.Conv2d(128, 256, 3, stride=2, padding=1),
                  nn.InstanceNorm2d(256),
                  nn.ReLU(inplace=True)]

        model += [nn.Conv2d(256, 256, 3, stride=2, padding=1),
                  nn.InstanceNorm2d(256),
                  nn.ReLU(inplace=True)]

        model += [nn.Conv2d(256, 512, 3, stride=2, padding=1),
                  nn.InstanceNorm2d(512),
                  nn.ReLU(inplace=True)]

        model += [nn.Conv2d(512, 1024, 3, stride=2, padding=1),
                  nn.InstanceNorm2d(1024),
                  nn.ReLU(inplace=True)]



        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


## Decoder

In [8]:
class Decoder(nn.Module):
    def __init__(self, input_nc=1024, output_nc=3):
        super(Decoder, self).__init__()
        model = [nn.ConvTranspose2d(input_nc, 512, 3, stride=2, padding=1, output_padding=1),
                  nn.InstanceNorm2d(512),
                  nn.ReLU(inplace=True)]

        model += [nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1, output_padding=1),
                  nn.InstanceNorm2d(256),
                  nn.ReLU(inplace=True)]
        
        model += [nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
                  nn.InstanceNorm2d(128),
                  nn.ReLU(inplace=True)]

        model += [nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
                  nn.InstanceNorm2d(64),
                  nn.ReLU(inplace=True)]

        model += [nn.ConvTranspose2d(64, 64, 3, stride=2, padding=1, output_padding=1),
                  nn.InstanceNorm2d(64),
                  nn.ReLU(inplace=True)]
                  
        
        # Output layer
        model += [  nn.ReflectionPad2d(3),
                    nn.Conv2d(64, output_nc, 7),
                    nn.InstanceNorm2d(3),
                    nn.Tanh() ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

## Identity Generator

In [9]:
class Identity_Generator(nn.Module):
    def __init__(self, encoder, decoder):
        super(Identity_Generator, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, A, B):
        latentA = self.encoder(A)
        latentB = self.encoder(B)

        reconstructedA = self.decoder(latentA)
        reconstructedB = self.decoder(latentB)
        return reconstructedA, reconstructedB

## Perceptual

In [10]:
class Perceptual(nn.Module):
    def __init__(self, encoder, decoder, generator):
        super(Perceptual, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.generator = generator

    def forward(self, A, B):

        reconstructedA, reconstructedB = self.generator(A, B)

        latentA = self.encoder(A)
        latentB = self.encoder(B)

        latentA.detach()
        latentB.detach()

        style = latentA[:, 0:512, : , :]
        content = latentB[:, 512:1024, :, :]
        
        mixed_latent = torch.cat([style, content], dim=1)
        mixed_image = self.decoder(mixed_latent)

        return mixed_image, reconstructedA, reconstructedB

## Discriminator

In [11]:
class Discriminator(nn.Module):
    def __init__(self, input_nc = 3):
        super(Discriminator, self).__init__()

        model = [   nn.Conv2d(input_nc, 64, 4, stride=2, padding=1),
                    nn.LeakyReLU(0.2, inplace=True) ]

        model += [  nn.Conv2d(64, 128, 4, padding=1),
                    nn.InstanceNorm2d(128), 
                    nn.LeakyReLU(0.2, inplace=True) ]

        model += [  nn.Conv2d(128, 256, 4, stride=2, padding=1),
                    nn.InstanceNorm2d(256), 
                    nn.LeakyReLU(0.2, inplace=True) ]

        model += [  nn.Conv2d(256, 512, 4, stride=2, padding=1),
                    nn.InstanceNorm2d(256), 
                    nn.LeakyReLU(0.2, inplace=True) ]

        model += [  nn.Conv2d(512, 1, 4, padding=1)]


        self.model = nn.Sequential(*model)

    def forward(self, x):
        x =  self.model(x)
        return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)

## Model test

In [12]:
test_tensor_source = torch.rand(20, 3, 64, 64)
test_tensor_target = torch.rand(20, 3, 64, 64)

encoder = Encoder()
decoder = Decoder()
generator = Identity_Generator(encoder, decoder)
perceptual = Perceptual(encoder, decoder, generator)
discriminator = Discriminator()

In [13]:
output = encoder(test_tensor_source)
print(output.size())

torch.Size([20, 1024, 2, 2])


In [14]:
output = decoder(output)
print(output.size())

torch.Size([20, 3, 64, 64])


In [15]:
output_a, output_b = generator(test_tensor_source, test_tensor_target)
print(output_a.size())
print(output_b.size())

torch.Size([20, 3, 64, 64])
torch.Size([20, 3, 64, 64])


In [16]:
mix, output_a, output_b = perceptual(test_tensor_source, test_tensor_target)

print(mix.size())
print(output_a.size())
print(output_b.size())

torch.Size([20, 3, 64, 64])
torch.Size([20, 3, 64, 64])
torch.Size([20, 3, 64, 64])


In [17]:
output = discriminator(test_tensor_source)
print(output.size())

torch.Size([20, 1])


In [18]:
print(mix[0][0][0] - output_a[0][0][0])

tensor([-0.3989,  0.1224,  0.1282, -1.1153,  0.1155, -1.2044,  0.2227, -0.4298,
         0.1723, -0.1405,  0.1651, -0.0550,  0.5790, -0.0945,  0.0311, -0.1396,
         0.2235,  1.2471,  0.1077, -0.4689, -1.5198,  0.8819, -0.3812,  0.5117,
        -0.6177, -0.8013,  0.2535, -0.3359, -0.0084,  0.4723, -0.9462,  0.0055,
         0.3436,  0.1828,  0.3792, -0.5670,  0.1498,  0.1871, -0.1718,  0.6821,
        -0.5080,  0.0293, -0.1309, -0.5036, -0.2128, -0.1984,  0.4339,  0.4402,
        -0.1365, -0.1834, -1.0720,  0.3227,  0.3862,  0.0127, -0.2784, -0.0785,
        -0.5141,  0.4760, -0.2940, -0.1618, -0.0534,  0.0352, -0.0706, -0.2237],
       grad_fn=<SubBackward0>)


# Loss

In [19]:
def tv_loss(img, tv_weight=5e-2):
    w_variance = torch.sum(torch.pow(img[:,:,:,:-1] - img[:,:,:,1:], 2))
    h_variance = torch.sum(torch.pow(img[:,:,:-1,:] - img[:,:,1:,:], 2))
    loss = tv_weight * (h_variance + w_variance)
    return loss

def total_variation_loss(img, weight=5e-2):
    bs_img, c_img, h_img, w_img = img.size()
    tv_h = torch.pow(img[:, :, 1:, :] - img[:, :, :-1, :], 2).sum()
    tv_w = torch.pow(img[:, :, :, 1:] - img[:, :, :, :-1], 2).sum()
    return weight * (tv_h + tv_w) / (bs_img * c_img * h_img * w_img)

def compute_content_loss(target_feature, content_feature):
    return torch.mean((target_feature - content_feature)**2)

def batch_gram_matrix(img):
    """
    Compute the gram matrix by converting to 2D tensor and doing dot product
    img: (batch, channel/depth, height, width)
    """
    b, d, h, w = img.size()
    img = img.view(b*d, h*w) # fix the dimension. It doesn't make sense to put b=1 when it's not always the case
    gram = torch.mm(img, img.t())
    return gram

style_weights = {'conv1_1': 1.,
                 'conv2_1': 0.75,
                 'conv3_1': 0.2,
                 'conv4_1': 0.2,
                 'conv5_1': 0.2}

def compute_style_loss(style_features, target_features):
    # the style loss
    # initialize the style loss to 0
    style_loss = 0
    style_grams = {layer: batch_gram_matrix(style_features[layer]) for layer in style_features}
    # then add to it for each layer's gram matrix loss
    for layer in style_weights:
        # get the "target" style representation for the layer
        target_feature = target_features[layer]
        target_gram = batch_gram_matrix(target_feature)
        _, d, h, w = target_feature.shape
        # get the "style" style representation
        style_gram = style_grams[layer]
        # the style loss for one layer, weighted appropriately
        layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram) ** 2)
        # add to the style loss
        style_loss += layer_style_loss / (d * h * w)

    return style_loss

# Training

## Training Method

In [20]:
def get_features(image, model, layers=None):
    if layers is None:
        layers = {'0': 'conv1_1',
                  '5': 'conv2_1',
                  '10': 'conv3_1',
                  '19': 'conv4_1',
                  '21': 'conv4_2',  ## content representation
                  '28': 'conv5_1'}

    features_arr = []
    features = {}
    x = image
    # model._modules is a dictionary holding each module in the model
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x

    return features

In [21]:
def train_with_autoencoder(source, target, preceptual, discriminator, criterion_adv, criterion_discriminator, criterion_construct, opt_pre, opt_dis, use_cuda, vgg, epoch):
    data_target_iter = iter(target)
    total = 0

    for batch_idx, (source_inputs, source_label) in enumerate(source):
        batch_size = source_inputs.size(0)
        total += batch_size
        target_inputs, target_label = data_target_iter.next()

        real_label = torch.ones(batch_size).float()
        fake_label = torch.zeros(batch_size).float()

        if use_cuda: 
            source_inputs, target_inputs = source_inputs.cuda(), target_inputs.cuda()
            real_label, fake_label = real_label.cuda(), fake_label.cuda()

        # preceptual
        mixed_image, reconstruct_source, reconstruct_target = preceptual(source_inputs, target_inputs)

        loss_ss = criterion_construct(source_inputs, reconstruct_source) * 30
        loss_tt = criterion_construct(target_inputs, reconstruct_target) * 30

        TV_loss = total_variation_loss(mixed_image)
        pred_fake = discriminator(mixed_image)
        loss_adv = criterion_adv(pred_fake, real_label)

        cuda_mixed_image = mixed_image.clone().requires_grad_(True).cuda()
        cuda_real_A = source_inputs.clone().requires_grad_(True).cuda()
        cuda_real_B = target_inputs.clone().requires_grad_(True).cuda()
        style_features = get_features(cuda_real_A, vgg)
        content_features = get_features(cuda_real_B, vgg)
        target_features = get_features(cuda_mixed_image, vgg)

        content_loss = compute_content_loss(target_features['conv4_2'], content_features['conv4_2']) * 0.1
        style_loss = compute_style_loss(style_features, target_features) * 0.05

        preceptual_loss = loss_ss + loss_tt + TV_loss + loss_adv + content_loss + style_loss

        opt_pre.zero_grad()
        preceptual_loss.backward()
        opt_pre.step()

        # Update discriminator
        pred_real = discriminator(source_inputs)
        pred_fake = discriminator(mixed_image.detach())

        loss_dis_real = criterion_discriminator(pred_real, real_label)
        loss_dis_fake = criterion_discriminator(pred_real, fake_label)

        discriminator_loss = loss_dis_real + loss_dis_fake

        opt_dis.zero_grad()
        discriminator_loss.backward()
        opt_dis.step()
    
        if  batch_idx % 200 == 0:
            real_A = source_inputs.data
            real_B = target_inputs.data
            mixed_image = mixed_image.data
            reconstructionA = reconstruct_source.data
            reconstructionB = reconstruct_target.data

            save_image(real_A, './output/%03d/00_%d_A.png' % ( epoch, batch_idx))
            save_image(real_B, 'output/%03d/00_%d_B.png' % ( epoch, batch_idx))
            save_image(reconstructionA, 'output/%03d/00_%d_reconA.png' % ( epoch, batch_idx))
            save_image(reconstructionB, 'output/%03d/00_%d_reconB.png' % ( epoch, batch_idx))
            save_image(mixed_image, 'output/%03d/00_%d_Mixed.png' % ( epoch, batch_idx))
        if batch_idx >= 400:
          break
    print ("e: %d, pre_loss: %.2f, D_real: %.2f, D_fake: %.2f" % (epoch, preceptual_loss, loss_dis_real, loss_dis_fake))


## Model hypermeter

In [22]:
class LambdaLR():
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert ((n_epochs - decay_start_epoch) > 0), "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch)

In [23]:
encoder = Encoder(input_nc = 3)
decoder = Decoder()
generator = Identity_Generator(encoder, decoder)
perceptual = Perceptual(encoder, decoder, generator)
discriminator = Discriminator()

In [24]:
learning_rate = 0.0002
beta = (0.5, 0.999)

criterion_adv = torch.nn.MSELoss()
criterion_discriminator = torch.nn.MSELoss()
criterion_construct = torch.nn.L1Loss()

optimizer_pre = torch.optim.Adam(perceptual.parameters(), lr=learning_rate, betas=beta)
optimizer_dis = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=beta)

lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_pre, lr_lambda=LambdaLR(200, 0, 100).step)
lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(optimizer_dis, lr_lambda=LambdaLR(200, 0, 100).step)


In [25]:
vgg = models.vgg19(pretrained=True).features
for param in vgg.parameters():
    param.requires_grad_(False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [26]:
if (torch.cuda.is_available()):
    torch.cuda.manual_seed_all(42)
    cudnn.benchmark = True

    criterion_construct = criterion_construct.cuda()
    criterion_discriminator = criterion_discriminator.cuda()
    criterion_construct = criterion_construct.cuda()
    
    encoder.cuda()
    decoder.cuda()
    generator.cuda()
    perceptual.cuda()
    discriminator.cuda()

## Init the network

In [27]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant(m.bias.data, 0.0)

In [28]:
encoder.apply(weights_init_normal)
decoder.apply(weights_init_normal)
generator.apply(weights_init_normal)
perceptual.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

## Training

In [ ]:
for epoch in range(0, 200):
    train_with_autoencoder(source = loader_target, 
        target = loader_source, 
        preceptual = perceptual, 
        discriminator = discriminator, 
        criterion_adv = criterion_adv, 
        criterion_discriminator = criterion_discriminator, 
        criterion_construct = criterion_construct, 
        opt_pre = optimizer_pre, 
        opt_dis = optimizer_dis, 
        use_cuda = True, 
        vgg = vgg,
        epoch = epoch)
    
    lr_scheduler_G.step()
    lr_scheduler_D.step()

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


e: 0, pre_loss: 10.94, D_real: 0.22, D_fake: 0.29
e: 1, pre_loss: 10.07, D_real: 0.23, D_fake: 0.27
e: 2, pre_loss: 11.52, D_real: 0.26, D_fake: 0.25
e: 3, pre_loss: 11.13, D_real: 0.27, D_fake: 0.23
e: 4, pre_loss: 12.78, D_real: 0.23, D_fake: 0.27
e: 5, pre_loss: 10.94, D_real: 0.33, D_fake: 0.19
e: 6, pre_loss: 11.39, D_real: 0.26, D_fake: 0.24
e: 7, pre_loss: 11.49, D_real: 0.17, D_fake: 0.35
e: 8, pre_loss: 11.80, D_real: 0.21, D_fake: 0.30
e: 9, pre_loss: 9.46, D_real: 0.32, D_fake: 0.19
e: 10, pre_loss: 10.33, D_real: 0.08, D_fake: 0.52
e: 11, pre_loss: 10.58, D_real: 0.29, D_fake: 0.22
e: 12, pre_loss: 9.87, D_real: 0.21, D_fake: 0.29
e: 13, pre_loss: 10.53, D_real: 0.28, D_fake: 0.22
e: 14, pre_loss: 10.88, D_real: 0.26, D_fake: 0.24
e: 15, pre_loss: 10.64, D_real: 0.26, D_fake: 0.24
e: 16, pre_loss: 9.85, D_real: 0.25, D_fake: 0.25
e: 17, pre_loss: 9.40, D_real: 0.30, D_fake: 0.21
e: 18, pre_loss: 11.53, D_real: 0.26, D_fake: 0.24
e: 19, pre_loss: 10.45, D_real: 0.31, D_fake:

# Reconstructed Image view

In [ ]:
def show_img(source, target, source_label, target_label):
    num_row = 4
    num_col = 5
    num = 10
    images = source[:num]
    labels = source_label[:num]

    fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
    for i in range(num):
        ax = axes[i//num_col, i%num_col]
        image =  images[i].transpose(0,2).transpose(0,1)

        ax.imshow(image)
        ax.set_title('Label: {}'.format(labels[i]))


    images = target[:num]
    labels = target_label[:num]
    for i in range(10,20):
        ax = axes[i//num_col, i%num_col]
        image = images[i - 10].transpose(0,2).transpose(0,1)
        
        ax.imshow(image)
        ax.set_title('Label: {}'.format(labels[i - 10]))
    plt.tight_layout()
    plt.show()

In [ ]:
def generate_image(source, target, preceptual, cuda):
    data_target_iter = iter(target)
    
    source_reconstrueted = []
    source_label_lst = []
    target_reconstructed = []
    target_label_lst = []
    mixed_image_lst = []
    for batch_idx, (source_inputs, source_label) in enumerate(source):
        target_inputs, target_label = data_target_iter.next()

        if cuda: 
            source_inputs, target_inputs = source_inputs.cuda(), target_inputs.cuda()

        # preceptual
        mixed_image, reconstruct_source, reconstruct_target = preceptual(source_inputs, target_inputs)

        source_reconstrueted.extend(reconstruct_source.data.cpu().detach())
        source_label_lst.extend(source_label.cpu().detach().numpy())
        target_reconstructed.extend(reconstruct_target.data.cpu().detach())
        target_label_lst.extend(target_label.cpu().detach().numpy())

        mixed_image_lst.extend(mixed_image.data.cpu().detach())

    return source_reconstrueted, target_reconstructed, source_label_lst, target_label_lst, mixed_image_lst
    

In [ ]:
'''source_reconstrueted, target_reconstructed, source_label, target_label, mixed = generate_image(source = loader_source, 
    target = loader_target, 
    preceptual = perceptual, 
    cuda = True)'''

In [ ]:

show_img(source_reconstrueted, target_reconstructed, source_label, target_label)

In [ ]:
num_row = 2
num_col = 5
num = 10
images = mixed[:num]

fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
for i in range(num):
    ax = axes[i//num_col, i%num_col]
        
    image =  images[i].transpose(0,2).transpose(0,1)

    ax.imshow(image)

## Training method

In [ ]:
'''def train(source, target, epoch, cuda = True):
     data_target_iter = iter(target)
     total = 0
     s_loss = 100000
     t_s_loss = 100000
     t_t_loss = 100000
     loss_min = 100000

     image_ss = []
     image_ts = []
     image_s = []
     image_t = []

     pre_source = None
     pre_target = None

     for batch_idx, (source_inputs, _) in enumerate(source):
          batch_size = source_inputs.size(0)
          total += batch_size
          optimizer_gen.zero_grad()

          target_inputs, _ = data_target_iter.next()

          if (pre_source == None and pre_target == None):
               pre_source = source_inputs
               pre_target = target_inputs

          source_inputs, target_inputs = Variable(source_inputs), Variable(target_inputs)
          
          if (cuda):
               source_inputs, target_inputs = source_inputs.cuda(), target_inputs.cuda()
               pre_source, pre_target = pre_source.cuda(), pre_target.cuda()

          reconstruct_source, reconstruct_target = generator(source_inputs, target_inputs)

          if(reconstruct_source.size()[0] != pre_target.size()[0]):
               break

          loss_construct_ss = criterion_construct(reconstruct_source, source_inputs)
          loss_construct_st = criterion_construct(reconstruct_source, pre_target)
          loss_construct_ts = criterion_construct(reconstruct_target, pre_source)
          loss_construct_tt = criterion_construct(reconstruct_target, target_inputs)


          total_loss = loss_construct_ss + loss_construct_st + loss_construct_ts + loss_construct_tt
          total_loss.backward()
          optimizer_gen.step()
          
          pre_source = reconstruct_source.detach()
          pre_target = reconstruct_target.detach()

          

          if (total_loss < loss_min):
               loss_min = total_loss

               image_ss = reconstruct_source.data.cpu().detach().numpy()
               image_ts = reconstruct_target.data.cpu().detach().numpy()
               image_t = target_inputs.data.cpu().detach().numpy()
               image_s = source_inputs.data.cpu().detach().numpy()



     return loss_min, image_ss, image_ts, image_t, image_s
'''

In [ ]:
'''for i in range(201):
    err, image_ss, image_ts, image_t, image_s = train(loader_source, loader_target, i)
    print("epoch: %d, error: %f" % (i, err))
    index = 0

    if(i % 10 == 0):
        fig, axs = plt.subplots(nrows=2, ncols = 2, figsize=(3, 5))
        img = image_s[index].reshape(28,28,3)
        img = (img + 1) * 0.5

        img2 = image_t[index].reshape(28,28,3)
        img2 = (img2 + 1) * 0.5

        img3 = image_ss[index].reshape(28,28,3)
        img3 = (img3 + 1) * 0.5

        img4 = image_ts[index].reshape(28,28,3)
        img4 = (img4 + 1) * 0.5

        axs[0, 0].imshow(img)
        axs[0 ,1].imshow(img2)
        axs[1 ,0].imshow(img3)
        axs[1 ,1].imshow(img4)

        fig.savefig('./reconstruct/%d_img.png' % i)'''

